# DATA LOAD
https://mimic.mit.edu/iii/
- ROW_ID :  simply a row identifier unique to that table
- SUBJECT_ID : a unique patient
- HADM_ID : a unique hospital admission

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
환자 = pd.read_csv('/project/data/폐렴환자.csv')
환자

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
1,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
2,251,268,F,2132-02-21 00:00:00,2198-02-18 00:00:00,2198-02-18 00:00:00,NaN,1
3,252,269,M,2130-09-30 00:00:00,NaN,NaN,NaN,0
4,628,665,M,2052-05-20 00:00:00,2120-02-04 00:00:00,2120-02-04 00:00:00,NaN,1
...,...,...,...,...,...,...,...,...
7802,31801,43932,M,2080-06-26 00:00:00,2153-03-01 00:00:00,NaN,2153-03-01 00:00:00,1
7803,31802,43937,M,2113-03-30 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,1
7804,31807,43959,F,2028-05-13 00:00:00,NaN,NaN,NaN,0
7805,31813,43982,F,2065-04-17 00:00:00,NaN,NaN,NaN,0


In [3]:
환자lab = pd.read_csv('/project/data/폐렴환자lab.csv')
환자lab

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,FLAG
0,5144,9,150750.0,50819,2149-11-14 06:07:00,NaN
1,5145,9,150750.0,50820,2149-11-14 06:07:00,NaN
2,5146,9,150750.0,50821,2149-11-14 06:07:00,abnormal
3,5147,9,150750.0,50826,2149-11-14 06:07:00,NaN
4,5148,9,150750.0,50827,2149-11-14 06:07:00,NaN
...,...,...,...,...,...,...
9356344,27427231,96442,120151.0,51248,2115-06-29 04:45:00,NaN
9356345,27427232,96442,120151.0,51249,2115-06-29 04:45:00,NaN
9356346,27427233,96442,120151.0,51250,2115-06-29 04:45:00,NaN
9356347,27427234,96442,120151.0,51254,2115-06-29 04:45:00,abnormal


# 중복 제거 

In [4]:
환자lab = 환자lab.drop(['ROW_ID','HADM_ID'],axis=1)
환자lab[환자lab.duplicated()]

,SUBJECT_ID,ITEMID,CHARTTIME,FLAG
5560,68,51476,2173-08-10 01:17:00,NaN
5566,68,51491,2173-08-10 01:17:00,NaN
5568,68,51492,2173-08-10 01:17:00,NaN
5571,68,51498,2173-08-10 01:17:00,NaN
5574,68,51508,2173-08-10 01:17:00,NaN
...,...,...,...,...
9110916,92644,51362,2114-04-27 12:04:00,abnormal
9110918,92644,51363,2114-04-27 12:04:00,NaN
9243999,98358,51266,2177-04-17 02:43:00,NaN
9276864,98358,51266,2177-03-31 04:10:00,NaN


In [5]:
print("제거할 데이터 개수:", 환자lab.duplicated().sum())
print("중복 제거 전 shape:", 환자lab.shape)

환자lab.drop_duplicates(inplace=True)
print("모두 중복되는 샘플 제거 후 shape:", 환자lab.shape)

제거할 데이터 개수: 6322
중복 제거 전 shape: (9356349, 4)
모두 중복되는 샘플 제거 후 shape: (9350027, 4)


# Nan 값을 문자열로

In [6]:
환자lab['FLAG'].isnull().sum()
환자lab['FLAG'] = 환자lab['FLAG'].fillna('nan')

In [7]:
# 전체 ITEMID의 개수 확인
print(환자lab['ITEMID'].nunique())
print('----------------')
# abnormal인 환자별로 ITEMID 개수 확인
환자lab[환자lab['FLAG'].str.contains('abnormal')].groupby(['SUBJECT_ID'])['ITEMID'].count()

690
----------------


SUBJECT_ID
9        113
21       646
31       149
36       665
37        91
        ... 
99863    320
99934     47
99937     46
99944     90
99985    305
Name: ITEMID, Length: 7799, dtype: int64

# 데이터가공

In [240]:
환자 = 환자[['SUBJECT_ID', 'EXPIRE_FLAG']]
환자lab = 환자lab[['SUBJECT_ID','ITEMID','FLAG']]

In [241]:
data = pd.merge(left = 환자 , right = 환자lab, how = "inner", on = "SUBJECT_ID")
data = data[['SUBJECT_ID','ITEMID','FLAG','EXPIRE_FLAG']]

In [242]:
data['FLAG'] = data['FLAG'].str.replace('delta', 'nan')
# data['FLAG'] = data['FLAG'].replace(['nan','abnormal'],[0,1])

In [243]:
환자id = data['SUBJECT_ID'].unique()

In [244]:
len(환자id)

7799

## 컬럼 itemid로 설정한 zero matrix 생성

In [245]:
itemid_uniq = list(data['ITEMID'].sort_values(ascending=True).unique())
itemid_uniq1 = list(map(str, itemid_uniq))
zero_matrix = np.zeros((len(환자id), len(itemid_uniq)))
df = pd.DataFrame(zero_matrix, columns=itemid_uniq1)

In [246]:
df['환자id'] = 환자id
df = df.set_index('환자id')

In [250]:
dfd = df.copy()
dfd

,50800,50801,50802,50803,50804,50805,50806,50808,50809,50810,...,51518,51519,51520,51521,51523,51525,51526,51529,51533,51537
환자id,,,,,,,,,,,,,,,,,,,,,
252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## abnormal만 추려서 zero matrix에 1 삽입

In [251]:
ab = data[data['FLAG']=='abnormal']
ab

,SUBJECT_ID,ITEMID,FLAG,EXPIRE_FLAG
4,252,51265,abnormal,0
5,252,51274,abnormal,0
7,252,51277,abnormal,0
8,252,51279,abnormal,0
15,252,50818,abnormal,0
...,...,...,...,...
9350006,44064,51279,abnormal,0
9350019,44064,51221,abnormal,0
9350020,44064,51222,abnormal,0
9350025,44064,51277,abnormal,0


In [253]:
ab.ITEMID

4          51265
5          51274
7          51277
8          51279
15         50818
           ...  
9350006    51279
9350019    51221
9350020    51222
9350025    51277
9350026    51279
Name: ITEMID, Length: 3404491, dtype: int64

In [254]:
for i, num in zip(ab['SUBJECT_ID'], ab.ITEMID):
#     print(i, num)
    dfd.loc[i, str(num)] = 1

In [255]:
dfd

,50800,50801,50802,50803,50804,50805,50806,50808,50809,50810,...,51518,51519,51520,51521,51523,51525,51526,51529,51533,51537
환자id,,,,,,,,,,,,,,,,,,,,,
252,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43932,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43937,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43959,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 잘 들어갔나 확인

In [257]:
data[data['SUBJECT_ID']==99937]

,SUBJECT_ID,ITEMID,FLAG,EXPIRE_FLAG
8451419,99937,51277,nan,1
8451420,99937,51279,abnormal,1
8451421,99937,51301,abnormal,1
8451422,99937,50868,nan,1
8451423,99937,50882,nan,1
...,...,...,...,...
8451656,99937,50912,nan,1
8451657,99937,50931,abnormal,1
8451658,99937,50954,abnormal,1
8451659,99937,50956,nan,1


In [256]:
dfd[dfd.index == 99937][['51279','51301','50931']]

,51279,51301,50931
환자id,,,
99937,1.0,1.0,1.0


## data에서 EXPIRE_FLAG loc해서 join

In [280]:
data1 = data.drop(['ITEMID','FLAG'],axis=1)
print(data1.shape)
data1.drop_duplicates(inplace=True)
data1.shape

(9350027, 2)


(7799, 2)

In [281]:
data1 = data1.set_index('SUBJECT_ID')
data1.index.name = '환자id'
process_data = dfd.add_prefix('ITEM_').jo   in(data1)

In [282]:
process_data

,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,...,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,
252,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
268,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43932,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
43937,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
43959,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 환자id를 다시 컬럼으로 지정하고 csv 저장

In [287]:
#환자id 기준으로 정렬
process_data = process_data.sort_values(by=['환자id']).reset_index()
process_data

,환자id,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,...,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,36,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,99863,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7795,99934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7796,99937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7797,99944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [291]:
process_data.to_csv('process_data.csv', index=False)

# 데이터가공 완료 - process_data.csv
- **value  = { 1 : abnormal, 0 : nan & delta }**
- **EXPIRE_FLAG = { 1 : 사망, 0: 생존(퇴원) }**

In [8]:
process_data = pd.read_csv('process_data.csv')
process_data

,환자id,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,...,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,36,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,99863,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7795,99934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7796,99937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7797,99944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [4]:
process_data['EXPIRE_FLAG'].value_counts()

1    4775
0    3024
Name: EXPIRE_FLAG, dtype: int64

In [9]:
process_data.sum(axis=1)

0           41.0
1           75.0
2           68.0
3           80.0
4           67.0
          ...   
7794     99899.0
7795     99960.0
7796     99959.0
7797     99966.0
7798    100035.0
Length: 7799, dtype: float64